<a href="https://colab.research.google.com/github/Bourbon-Rye/Baesian-Cropability/blob/main/PilipiNuts_2023_Baesian_Cropability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Note: See Hypothesis Testing RESULTS in `results/`.

Set `INTERACTIVE` global variable (below) to <span style="color:blue">True</span> for interactive plots and <span style="color:blue">False</span> for static plots.

In [27]:
# @Libraries
import numpy as np
import pandas as pd
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
import json
import re
import functools

from pathlib import Path
from sklearn import preprocessing
from plotly.subplots import make_subplots
from plotly import offline
from scipy import stats
from sklearn.impute import SimpleImputer
from matplotlib import figure

INTERACTIVE = True

def render(fig: figure.Figure):
    if INTERACTIVE:
        fig.show()
    else:
        fig.update_layout(
            autosize=False,
            width=1280,
            height=720,
        )
        fig.show(renderer="png")

In [28]:
# @title Corrections
with open('datasets/region_provinces.json') as jsonfile:
    regions_provinces = json.load(jsonfile)['PHILIPPINES']
regions_provinces = {key.lower():regions_provinces[key] for key in regions_provinces}
regions = regions_provinces.keys()
provinces = set([item.lower() for key in regions for item in regions_provinces[key]])
regions = set(regions)
regions.add("philippines")

# NOTE: Fix for bad regions, thanks PSA
bad_regions = ['AONCR', 'BARMM', 'CAR', 'MIMAROPA', 'NCR',
           'Region 1', 'Region 2', 'Region 3', 'Region 4A',
           'Region 5', 'Region 6', 'Region 7', 'Region 8',
           'Region 9', 'Region 10', 'Region 11', 'Reg12', 'CARAGA']
region_mapping = {bad.lower():good.lower() for (bad,good) in zip(bad_regions, regions_provinces.keys())}
corrections = {
    "AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)": "bangsamoro autonomous region in muslim mindanao (barmm)",
    "autonomous region in muslim mindanao (armm)": "bangsamoro autonomous region in muslim mindanao (barmm)",
    "mimaropa region": "mimaropa region (mimaropa)"
}
region_mapping.update(corrections)

temp = [key.split('(')[1].rstrip(')') for key in regions_provinces]
region_short_to_long = {bad:good for (bad,good) in zip(temp, regions_provinces.keys())}
region_long_to_short = {v: k for k, v in region_short_to_long.items()}

In [29]:
# @title Utilities
def read_csv_to_df(csvfile: Path, comment_symbol='#') -> pd.DataFrame:
    """Read CSV to DataFrame with comment validation.
        Allows comment lines in CSVs where line[0] == comment_symbol.
        Also removes newlines.
        Does not recognize comment_symbol anywhere else. 
    """
    tempfile = Path('temp.csv')
    with open(csvfile, 'r') as csv, open(tempfile, 'w+') as temp:
        lines = csv.readlines()
        for line in lines:
            if line[0] != comment_symbol:
                temp.write(line)
    return pd.read_csv(tempfile)

def is_region(x: str, regions=regions) -> bool:
    x = x.strip(' .')
    x = x.lower()
    if x == "cagayan":
        return False
    for region in regions:
        if x in region:
            return True
    else:
        return False
    
def is_province(x: str, provinces=provinces) -> bool:
    x = x.strip(' .')
    x = x.lower()
    for province in provinces:
        if x in province:
            return True
    else:
        return False

def get_quarter_columns(df: pd.DataFrame, year_range: range, period_idx: int):
    """Assumes contiguous period (Year Month) columns and that columns before period_idx are ID columns"""
    df_quarter = df.iloc[:, :period_idx].copy()
    for year in year_range:
        for q in range(0, 12, 3):
            df_quarter[f"{year} Q{q//3+1}"] = df.filter(regex=str(year), axis=1).iloc[:, q:q+3].mean(axis=1)
    return df_quarter

def get_annual_columns(df: pd.DataFrame, year_range: range, period_idx: int):
    """Assumes contiguous period (Year Month|Quarter) columns and that columns before period_idx are ID columns
    Note that this also works with Quarters"""
    df_annual = df.iloc[:, :period_idx].copy()
    for year in year_range:
        df_annual[f"{year}"] = df.filter(regex=str(year), axis=1).mean(numeric_only=True, axis=1)
    return df_annual

def swap_columns(df: pd.DataFrame, col1: str, col2: str):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

def normalize(df: pd.DataFrame, col: str, minmax = True):
    """Can use mean normalization and minmax normalization."""
    tdf = df[col]
    if minmax:
        tdf = (tdf-tdf.min())/(tdf.max()-tdf.min())
    else:
        tdf = (tdf-tdf.mean())/tdf.std()
    df[col] = tdf
    return df

def drop_rows_with_zeros(df: pd.DataFrame, ref_col_idx: int, all_zeros=False):
    """Drop rows if some values are zeros, or if all values are zeros.
    Assumes contiguous reference columns, i.e. columns to use in deciding whether to drop."""
    return df[~(df.iloc[:, ref_col_idx:] == 0).all(axis=1)] if all_zeros else df[~(df.iloc[:, ref_col_idx:] == 0).any(axis=1)]

def dual_plot(df: pd.DataFrame, x: str, y1: str, y2: str,
              title=None, xtitle=None, y1title=None, y2title=None):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Add traces
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y1], name=y1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y2], name=y2),
        secondary_y=True,
    )
    # # Add titles
    if title is not None: fig.update_layout(title_text=title)
    if xtitle is not None: fig.update_xaxes(title_text=xtitle)
    if y1title is not None: fig.update_yaxes(title_text=y1title, secondary_y=False)
    if y2title is not None: fig.update_yaxes(title_text=y2title, secondary_y=True)

    return fig


def triplet_plot(df: pd.DataFrame, x: str, y1_1: str, y1_2: str, y2: str,
                 title=None, xtitle=None, y1title=None, y2title=None):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Add traces
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y1_1], name=y1_1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y1_2], name=y1_2),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y2], name=y2),
        secondary_y=True,
    )
    # # Add titles
    if title is not None: fig.update_layout(title_text=title)
    if xtitle is not None: fig.update_xaxes(title_text=xtitle)
    if y1title is not None: fig.update_yaxes(title_text=y1title, secondary_y=False)
    if y2title is not None: fig.update_yaxes(title_text=y2title, secondary_y=True)

    return fig

def quartet_plot(df: pd.DataFrame, x: str, y1_1: str, y1_2: str, y2_1: str, y2_2: str,
                 title=None, xtitle=None, y1title=None, y2title=None):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Add traces
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y1_1], name=y1_1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y1_2], name=y1_2),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y2_1], name=y2_1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y2_2], name=y2_2),
        secondary_y=True,
    )
    # # Add titles
    if title is not None: fig.update_layout(title_text=title)
    if xtitle is not None: fig.update_xaxes(title_text=xtitle)
    if y1title is not None: fig.update_yaxes(title_text=y1title, secondary_y=False)
    if y2title is not None: fig.update_yaxes(title_text=y2title, secondary_y=True)

    return fig

def move_column(df: pd.DataFrame, col: str, new_idx: int):
    """This is an inplace method."""
    df.insert(new_idx, col, df.pop(col))

def imputer(df: pd.DataFrame, start_idx: int, end_idx=None):
    """If only start index is provided, will impute from start index column to last column,
    else limit from start_idx to end_idx. Inplace imputation"""
    imp = SimpleImputer(missing_values=pd.NA, strategy='mean')
    if end_idx:
        imp.fit(df.iloc[:, start_idx:end_idx])
        df[df.columns[start_idx:end_idx]] = imp.transform(df.iloc[:, start_idx:])
    else:
        imp.fit(df.iloc[:, start_idx:])
        df[df.columns[start_idx:]] = imp.transform(df.iloc[:, start_idx:])

def preprocess_baesians_2(df: pd.DataFrame, commodity_map: dict, melt_value: str|None, regional=True, impute=False):
    """Assumes Geolocation | Commodity | Period ... columns.
    Filters to regional if regional=True, else filters to provincial. Retains "philippines".
    Filters to raw commodity name (key in commodity_map), and then renames to standard commodity_map[key]
    Ex. {"RICE, REGULAR-MILLED, 1 KG" : "Rice"}
    """
    df["Geolocation"] = df["Geolocation"].str.lstrip(".").str.lower()
    df["Geolocation"] = df["Geolocation"].replace(region_mapping)
    df = df[df["Geolocation"].apply(is_region)] if regional else df[df["Geolocation"].apply(is_province)]
    df = df[df["Commodity"].isin(commodity_map)]
    df.loc[:, "Commodity"] = df["Commodity"].replace(commodity_map)
    if melt_value:
        df = df.melt(id_vars=["Geolocation", "Commodity"], value_vars=df.columns[2:], var_name="Period", value_name=melt_value)
    if impute:
        imputer(df, 3)  # inplace imputation of melt_value
    return df

def filter_to_regions(df: pd.DataFrame):
    return df[df["Geolocation"].apply(is_region)]
    
def filter_to_provinces(df: pd.DataFrame):
    return df[df["Geolocation"].apply(is_province)]

def fig_to_div(fig: figure.Figure, filename: Path):
    """Optional: Add pretiffication."""
    filename = Path(filename)
    filename.parent.mkdir(exist_ok=True, parents=True)
    with open(filename, "w+") as f:
        f.write(offline.plot(fig, include_plotlyjs=False, output_type='div'))
        
def fig_to_html(fig: figure.Figure, filename: Path):
    filename = Path(filename)
    filename.parent.mkdir(exist_ok=True, parents=True)
    fig.write_html(filename)
    
def fig_to_png(fig: figure.Figure, filename: Path, transparent=False):
    """NOTE: Requires ORCA to be installed!"""
    filename = Path(filename)
    filename.parent.mkdir(exist_ok=True, parents=True)
    if transparent:
        fig.update_layout({
            "plot_bgcolor": "rgba(0, 0, 0, 0)",
            "paper_bgcolor": "rgba(0, 0, 0, 0)",
        })
    pio.write_image(fig, filename,scale=6, width=900, height=680)
    
def p_report(p: int) -> str|int:
    if p >= 0.001:
        return round(p, 3)
    elif p < 0.001:
        return "<0.001"
    else:
        return np.nan
    

In [30]:
# ANNUAL RICE and CORN STOCKS (5/24/2024) NOTE: Perhaps try to match this with the prices as this has monthly, for national only
df = read_csv_to_df("datasets/agricultural-indicators/stocks-palay-corn_yearly_1980-2024.csv")
df = df[(df["Sector"] == "Rice: Total Stock") | (df["Sector"] == "Corn: Total Stock")]
df["Sector"] = df["Sector"].replace({"Rice: Total Stock": "Rice", "Corn: Total Stock": "Corn"})
df = pd.concat([df.iloc[:, :2].copy(), df.mean(axis=1, numeric_only=True)], axis=1)
df.rename(columns={"Sector": "Commodity", "Year": "Period", 0: "Stocks"}, inplace=True)
df = df[df["Period"].isin(range(2012, 2024))]
df_stocks = df

# Baesian Plots


Hypotheses:
- **H0.1:** There is no significant difference in the productivity of major food crops when grouped according to their crop type, geolocation, and/or market profile.
- **H0.2:** There is no significant relationship between market conditions and food crop production.

Goal: Visually and statistically assess the relationship between market indicators and crop yield indicators.

Visual tests are dual plots and scatterplots. Statistics for relationship testing: contingency tables and t-tests (?).

## Data Overview

Selected major crops (analyzed are checked):
- ✅ Rice / Palay
- ✅ Corn / Maize
- Sweet Potato / Camote
- Mongo / Monggo / Mung Beans
- Banana
- Coconut
- 🚧 Onion
- 🚧 Garlic
- Sugarcane

In [31]:
# Volume of Production of Selected Major Crops (Philippines)
df = pd.read_csv("datasets/agricultural-indicators/volume_rice-corn.csv", skiprows=2)
df = df[df["Geolocation"] == "PHILIPPINES"]
df = df[(df["Commodity"] == "Palay") | (df["Commodity"] == "Corn")]
df = df.filter(regex="Commodity|Geolocation|Annual", axis=1)
df.columns = map(lambda x: x.replace(" Annual", "") if "Annual" in x else x, df.columns)
df = df.melt(id_vars=["Commodity", "Geolocation"], value_vars=df.columns[2:], var_name="Period", value_name="Volume")
fig = px.histogram(df, x="Period", y="Volume", color="Commodity", barmode='group',
                   title='Annual Volume of Production of Rice and Corn<br><sup>National averages in metric tons</sup>').update_layout(
    xaxis_title="Period", yaxis_title = "Volume of Production", template="plotly_dark")
render(fig)

## Annual Analysis
Annual-National and Annual-Regional analysis of the commodities Rice and Corn. Other major crops to follow.

**NOTE:** The core agricultural indicators are Stocks, Volume, and Area Harvested, while the core market indicators are Farmgate Price, Wholesale Price, and Retail Price.

This is important to know especially during visualization where Farmgate price is used as a slicer.

### Data Preprocessing

In [32]:
# @title Annual megadataset for rice and corn
# NOTE: Filter this during visualization and analysis to just Philippines or Regions
def filter_period(df: pd.DataFrame, _type: str):
    if _type == "Annual":
        df = df.filter(regex="Commodity|Geolocation|Annual", axis=1)
        df.columns = map(lambda x: x.replace(" Annual", "") if "Annual" in x else x, df.columns)
    elif _type == "Quarterly":
        df = df.filter(regex="Commodity|Geolocation|Q\d", axis=1)
    return df

# Volume of Rice and Corn
df1 = pd.read_csv("datasets/agricultural-indicators/volume_rice-corn.csv", skiprows=2, na_values=[".."])
df1 = filter_period(df1, "Annual")
df1 = preprocess_baesians_2(df1, {"Palay": "Rice", "Corn": "Corn"}, melt_value="Volume")

# Farmgate Price of Rice and Corn
df2 = pd.read_csv("datasets/prices/prices_farmgate-new-series_2010-2023.csv")
df2 = get_annual_columns(df2, range(2012, 2024), 2)
df2 = preprocess_baesians_2(df2, {"Palay [Paddy] Other Variety, dry (conv. to 14% mc)": "Rice",
                                  "Corngrain [Maize] Yellow, matured": "Corn"}, melt_value="Farmgate Price")

# Wholesale Price of Rice and Corn
df3 = pd.read_csv("datasets/prices/prices_wholesale-new-series_2010-2023.csv",)
df3 = get_annual_columns(df3, range(2012, 2024), 2)
df3 = preprocess_baesians_2(df3, {"Well Milled Rice (WMR)": "Rice",
                                  "Corngrits White": "Corn"}, melt_value="Wholesale Price")

# Retail Price of Rice and Corn
df4 = pd.read_csv("datasets/prices/prices_retail_2012-2023.csv")
df4 = get_annual_columns(df4, range(2012, 2024), 2)
df4 = preprocess_baesians_2(df4, {"RICE, REGULAR-MILLED, 1 KG": "Rice",
                                  "RICE, REGULAR MILLED, LOOSE, 1 KG" : "Rice",
                                  'WHOLE CORN GRAIN, YELLOW, 1 KG': "Corn"}, melt_value="Retail Price")

# Area Harvested of Rice and Corn
df5 = pd.read_csv("datasets/agricultural-indicators/area-harvested-palay-corn_quarterly-annual_2010-2023.csv", skiprows=2)
df5 = filter_period(df5, "Annual")
df5 = preprocess_baesians_2(df5, {"Palay": "Rice", "Corn": "Corn"}, melt_value="Area Harvested")

# Consumer Price Index (All Income) per Region of Rice and Corn
df6 = pd.read_csv("datasets/price-indices-2018-based/cpi_all-income-households-by-cg-with-backcasting_1994-2023.csv")
df6 = get_annual_columns(df6, range(2012, 2024), 2)
df6 = preprocess_baesians_2(df6, {"01.1.1.12 - Rice": "Rice",
                                  "01.1.1.16 - Corn": "Corn"},
                            melt_value="CPI All Income")

# Consumer Price Index (Bottom 30) per Region of Rice and Corn
df7 = pd.read_csv("datasets/price-indices-2018-based/cpi_bottom-30-by-cg-with-backcasting_2012-2017.csv")
df7 = preprocess_baesians_2(df7, {"01.1.1.12 - Rice": "Rice", "01.1.1.16 - Corn": "Corn"}, melt_value=None)
tdf = pd.read_csv("datasets/price-indices-2018-based/cpi_bottom-30-by-cg_2018-2023.csv")
tdf = preprocess_baesians_2(tdf, {"01.1.1.12 - Rice": "Rice", "01.1.1.16 - Corn": "Corn"}, melt_value=None)
df7 = pd.merge(df7, tdf, on=["Geolocation", "Commodity"])
df7 = get_annual_columns(df7, range(2012, 2024), 2)
df7 = preprocess_baesians_2(df7, {"Rice": "Rice", "Corn": "Corn"}, melt_value="CPI Bottom 30")

# Costs and Returns per Region of Rice and Corn
df8 = pd.read_csv("datasets/agricultural-indicators/costs-and-returns_rice-and-corn.csv", skiprows=1, na_values=[".."])
df8 = df8[df8["Item"] == "NET RETURNS"]
df8.drop("Item", axis=1, inplace=True)
df8.rename({"Type": "Commodity"}, axis=1, inplace=True)
df8 = preprocess_baesians_2(df8, {"All Palay": "Rice", "All Corn": "Corn"}, melt_value="Net Returns")
df8 = df8[df8["Period"].str.contains("Average")]
df8["Period"] = df8["Period"].apply(lambda x: x.split()[1])

# National Inflation Rate
df9 = pd.read_csv("datasets/statista_inflation-rate-in-the-philippines-2029.csv", dtype={"Period": object, "Inflation Rate": float})

# NCR Retail Price Index on Food
df10 = pd.read_csv("datasets/price-indices-2018-based/rpi-in-ncr_food-only_1998-2023.csv")
df10 = get_annual_columns(df10, range(2010, 2024), 2)
df10 = preprocess_baesians_2(df10, {"Food": "Food"}, melt_value="NCR RPI").drop(["Geolocation", "Commodity"], axis=1)

# Agricultural Self-Sufficiency for Rice and Corn
df11 = pd.read_csv("datasets/agricultural-indicators/agri-self-sufficiency-ratio.csv", skiprows=1)
df11 = df11[(df11["Commodity"] == "Rice") | (df11["Commodity"] == "Corn")]
df11 = df11.melt("Commodity", df11.columns[1:], "Period", "Self-Sufficiency Ratio")

# Agricultural Import-Dependency for Rice and Corn
df12 = pd.read_csv("datasets/agricultural-indicators/agri-import-dependency-ratio.csv", skiprows=1)
df12 = df12[(df12["Commodity"] == "Rice") | (df12["Commodity"] == "Corn")]
df12 = df12.melt("Commodity", df12.columns[1:], "Period", "Import-Dependency Ratio")

# SU Gross Supply and UT Total Net Food Disposable for Rice and Corn
# Explained here: https://openstat.psa.gov.ph/Metadata/2B5FSUA0
df13 = pd.read_csv("datasets/agricultural-indicators/supply-ut_rice-corn_1990-2022.csv")
df13 = df13.filter(regex="Commodity|Year|Gross|Food")
df13.rename({"Year": "Period", "UT Total Net Food Disposable": "UT Consumable"}, axis=1, inplace=True)
df13.Period = df13.Period.astype(str)

# Merge all dfs into a single df
# NOTE: Remove df8 or add more year data or commodities to increase number of samples
# NOTE: You may adjust here which dfs are included in the final df
# NOTE: df8, df10, df11, df12, and df13 are all only until 2021 and 2022... respectively
UNTIL_2023 = False
if UNTIL_2023:
    dfs = [df1, df2, df3, df4, df5, df6, df7, df9]
    df = functools.reduce(lambda left, right: pd.merge(left, right), dfs)
else:
    dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]
    df = functools.reduce(lambda left, right: pd.merge(left, right), dfs)
    move_column(df, "Area Harvested", 4)
    move_column(df, "Self-Sufficiency Ratio", 5)
    move_column(df, "Import-Dependency Ratio", 6)
    move_column(df, "SU Gross Supply", 7)
    move_column(df, "UT Consumable", 8)

# # Stocks of Rice and Corn (Annual)
tdf1 = df_stocks.copy()
tdf2 = df[["Commodity", "Period"]].copy()
tdf2["Period"] = pd.to_numeric(tdf2["Period"])
df.insert(3, "Stocks", pd.merge(tdf2, tdf1)["Stocks"])

# Cleanup
del dfs, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13

if False:
    df.to_csv("datasets/annual-regional_megadataset_inner-join.csv", index=False)
    
display(df.describe())
display(df[["Geolocation", "Commodity", "Period"]].describe())

display(df.groupby("Geolocation").size())
df.groupby("Geolocation").size()
df

<>:8: DeprecationWarning:

invalid escape sequence '\d'



,Stocks,Volume,Area Harvested,Self-Sufficiency Ratio,Import-Dependency Ratio,SU Gross Supply,UT Consumable,Farmgate Price,Wholesale Price,Retail Price,CPI All Income,CPI Bottom 30,Net Returns,Inflation Rate,NCR RPI
count,340.000000,3.400000e+02,3.400000e+02,340.00000,340.00000,340.000000,340.00000,330.000000,252.000000,244.000000,340.000000,340.000000,305.000000,340.000000,340.000000
mean,1485.030538,1.558783e+06,4.275902e+05,91.11000,8.89500,12609.000000,7286.35000,15.157641,33.806509,35.386332,92.842937,93.060681,16309.521311,2.813000,112.559167
std,892.227575,3.314457e+06,8.629677e+05,4.80195,4.80107,3935.472969,5210.44233,2.901644,6.415134,8.506258,10.446669,10.317620,10258.809888,1.251995,8.668396
min,320.012308,5.865700e+04,2.449600e+04,79.80000,1.80000,7710.000000,1569.00000,9.174167,18.000000,12.994167,66.711141,62.617408,-1775.000000,0.690000,100.000000
25%,649.277692,2.589409e+05,1.031898e+05,88.77500,5.35000,8707.750000,2164.00000,12.499861,28.584375,33.198750,87.044130,86.623208,9355.000000,2.390000,104.683333
50%,1544.326538,5.673236e+05,1.639720e+05,91.95000,8.10000,12360.500000,7029.00000,15.137083,36.015455,38.064167,93.423984,93.606207,15782.000000,2.720000,112.279167
75%,2267.125577,1.287862e+06,3.745478e+05,94.65000,11.22500,16071.500000,11706.50000,17.455208,38.871458,40.611250,97.503626,97.956751,22686.000000,3.590000,120.500000
max,2819.688462,1.996017e+07,4.811808e+06,98.20000,20.20000,18353.000000,14886.00000,21.994167,46.368333,62.883333,168.191370,165.011797,57587.000000,5.310000,125.858333


,Geolocation,Commodity,Period
count,340,340,340
unique,17,2,10
top,philippines,Rice,2012
freq,20,170,34


Geolocation
bangsamoro autonomous region in muslim mindanao (barmm)    20
cordillera administrative region (car)                     20
mimaropa region (mimaropa)                                 20
philippines                                                20
region i (ilocos region)                                   20
region ii (cagayan valley)                                 20
region iii (central luzon)                                 20
region iv-a (calabarzon)                                   20
region ix (zamboanga peninsula)                            20
region v (bicol region)                                    20
region vi (western visayas)                                20
region vii (central visayas)                               20
region viii (eastern visayas)                              20
region x (northern mindanao)                               20
region xi (davao region)                                   20
region xii (soccsksargen)                                 

,Geolocation,Commodity,Period,Stocks,Volume,Area Harvested,Self-Sufficiency Ratio,Import-Dependency Ratio,SU Gross Supply,UT Consumable,Farmgate Price,Wholesale Price,Retail Price,CPI All Income,CPI Bottom 30,Net Returns,Inflation Rate,NCR RPI
0,philippines,Rice,2012,2227.093846,18032525.47,4690061.17,91.9,8.1,15465,11473,15.923333,32.800833,33.476667,81.632653,82.987552,19891.0,3.16,100.000000
1,cordillera administrative region (car),Rice,2012,2227.093846,453461.00,120100.00,91.9,8.1,15465,11473,16.721667,32.416667,35.628333,85.616438,85.034014,7274.0,3.16,100.000000
2,region i (ilocos region),Rice,2012,2227.093846,1737695.00,403169.00,91.9,8.1,15465,11473,17.065000,31.941667,31.070000,79.113924,79.617834,21639.0,3.16,100.000000
3,region ii (cagayan valley),Rice,2012,2227.093846,2425536.47,582557.17,91.9,8.1,15465,11473,16.246667,30.702500,31.335833,80.971660,83.752094,16638.0,3.16,100.000000
4,region iii (central luzon),Rice,2012,2227.093846,3220607.00,675781.00,91.9,8.1,15465,11473,17.145000,32.866667,31.469167,79.617834,79.808460,30333.0,3.16,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,region x (northern mindanao),Corn,2021,794.027692,1455030.45,384864.96,94.8,5.2,9661,2609,14.165000,29.346667,24.740833,100.408755,100.126349,29280.0,3.93,125.858333
336,region xi (davao region),Corn,2021,794.027692,266894.04,175295.00,94.8,5.2,9661,2609,12.265000,30.242500,20.895000,90.597157,90.246810,NaN,3.93,125.858333
337,region xii (soccsksargen),Corn,2021,794.027692,1105238.85,397363.00,94.8,5.2,9661,2609,15.142500,30.406667,NaN,102.613642,102.581773,14274.0,3.93,125.858333
338,region xiii (caraga),Corn,2021,794.027692,141648.84,40933.00,94.8,5.2,9661,2609,12.490833,32.963333,NaN,115.890568,115.778763,15564.0,3.93,125.858333


## Data Modeling

In [33]:
# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('once')

print('Skforecast version: ', skforecast.__version__)

Skforecast version:  0.12.1


In [34]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.model_selection import TimeSeriesSplit

df: pd.DataFrame = pd.read_csv(
    "datasets/agricultural-indicators/stocks-palay-corn_yearly_1980-2024.csv"
)
df = df[(df["Sector"] == "Rice: Total Stock") | (df["Sector"] == "Corn: Total Stock")]
df["Sector"] = df["Sector"].replace(
    {"Rice: Total Stock": "Rice", "Corn: Total Stock": "Corn"}
)
df = df[df["Sector"] == "Rice"]
df = df.melt(["Sector", "Year"], df.columns[2:], "Month", "Stocks")
df["Year"] = df["Year"].astype(str)
df["Period"] = df["Month"] + " " + df["Year"]
df["Period"] = pd.to_datetime(df["Period"], format="%b %Y")
df.drop(["Year", "Month"], axis=1, inplace=True)
df = df.set_index("Period")
df = df.asfreq("MS")
df = df.sort_index()
imp_mean = KNNImputer()
df["Stocks"] = imp_mean.fit_transform(pd.DataFrame(df["Stocks"]))
df = df[df.index < "2024"]

# We allow a break in the
steps = 52
data_train = df[:-steps]
data_test = df[-steps:]
print(
    f"Train dates : {data_train.index.min()} --- "
    f"{data_train.index.max()}  (n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- "
    f"{data_test.index.max()}  (n={len(data_test)})"
)

# display(df)
from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=123,
    loss='squared_error'
)
forecaster = ForecasterAutoreg(
    regressor=est, lags=6
)
forecaster.fit(y=data_train["Stocks"])
forecaster

predictions = forecaster.predict(steps=steps)
predictions.head(5)

# Plot predictions versus test data
# ==============================================================================

fig = make_subplots()
fig.add_scatter(
    x=data_train.index, y=data_train.Stocks, name=f"train\t(n={len(data_train)})"
)
fig.add_scatter(
    x=data_test.index, y=data_test.Stocks, name=f"test\t\t(n={len(data_test)})"
)
fig.add_scatter(
    x=predictions.index, y=predictions, name="predictions", line=dict(color="#FFA500")
)
fig.update_layout(
    legend=dict(
        yanchor="top", y=0.98, xanchor="left", x=0.01, bgcolor="rgba(0, 0, 0, 0)"
    ),
    title="Philippines Total Rice Stocks (1980-2023)<br>"
    "<sup>Sum of Household, Commercial, and NFA stocks</sup>",
    yaxis=dict(showgrid=False, title="Stocks (mt)"),
    xaxis=dict(showgrid=False, title="Period"),
)
render(fig)

error_mse = mean_squared_error(
                y_true = data_test['Stocks'],
                y_pred = predictions
            )
print(f"Test error (MSE): {error_mse}")

error_rmse = root_mean_squared_error(
                y_true = data_test['Stocks'],
                y_pred = predictions
            )
print(f"Test error (RMSE): {error_rmse}")



Train dates : 1980-01-01 00:00:00 --- 2019-08-01 00:00:00  (n=476)
Test dates  : 2019-09-01 00:00:00 --- 2023-12-01 00:00:00  (n=52)


Test error (MSE): 223039.73980227724
Test error (RMSE): 472.2708331056209


In [35]:
# Hyperparameters: grid search
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = GradientBoostingRegressor(random_state=123),
                 lags      = 12 # This value will be replaced in the grid search
             )

# Candidate values for lags
lags_grid = [10, 20]

# # Candidate values for regressor's hyperparameters
# param_grid = {
#     'n_estimators': [100, 250],
#     'max_depth': [3, 5, 10],
#     'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2],
#     'criterion': ['friedman_mse', 'squared_error'],
# }

# results_grid = grid_search_forecaster(
#                    forecaster         = forecaster,
#                    y                  = data_train['Stocks'],
#                    param_grid         = param_grid,
#                    lags_grid          = lags_grid,
#                    steps              = steps,
#                    refit              = False,
#                    metric             = 'mean_squared_error',
#                    initial_train_size = int(len(data_train)*0.5),
#                    fixed_train_size   = False,
#                    return_best        = True,
#                    n_jobs             = 'auto',
#                    verbose            = False
#                )


In [36]:
regressor = RandomForestRegressor(n_estimators=250, max_depth=10, random_state=123)
est = GradientBoostingRegressor(
    criterion="squared_error",
    learning_rate=0.1,
    loss="huber",
    max_depth=10,
    n_estimators=250,
    random_state=123,
)
forecaster = ForecasterAutoreg(regressor=regressor, lags=20)
forecaster.fit(y=data_train["Stocks"])
predictions = forecaster.predict(steps=steps)

fig = make_subplots()
fig.add_scatter(
    x=data_train.index, y=data_train.Stocks, name=f"train\t(n={len(data_train)})"
)
fig.add_scatter(
    x=data_test.index, y=data_test.Stocks, name=f"test\t\t(n={len(data_test)})"
)
fig.add_scatter(
    x=predictions.index, y=predictions, name="predictions", line=dict(color="#FFA500")
)
fig.update_layout(
    legend=dict(
        yanchor="top", y=0.98, xanchor="left", x=0.01, bgcolor="rgba(0, 0, 0, 0)"
    ),
    title="Philippines Total Rice Stocks (1980-2023)<br>"
    "<sup>Sum of Household, Commercial, and NFA stocks</sup>",
    yaxis=dict(showgrid=False, title="Stocks (mt)"),
    xaxis=dict(showgrid=False, title="Period"),
)
render(fig)

error_mse = mean_squared_error(y_true=data_test["Stocks"], y_pred=predictions)
print(f"Test error (MSE): {error_mse}")

error_rmse = root_mean_squared_error(y_true=data_test["Stocks"], y_pred=predictions)
print(f"Test error (RMSE): {error_rmse}")


Test error (MSE): 62203.20123843953
Test error (RMSE): 249.40569608258656


In [37]:
# tscv = TimeSeriesSplit()
# splits = list(tscv.split(df))
# for train, test in splits:
#     print(train, test)
    
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import HistGradientBoostingRegressor
# from sklearn.model_selection import cross_validate
# from sklearn.pipeline import make_pipeline

# gbrt = HistGradientBoostingRegressor(categorical_features="from_dtype", random_state=42)
# categorical_columns = X.columns[X.dtypes == "category"]
# print("Categorical features:", categorical_columns.tolist())
    
# import numpy as np


# def evaluate(model, X, y, cv, model_prop=None, model_step=None):
#     cv_results = cross_validate(
#         model,
#         X,
#         y,
#         cv=cv,
#         scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
#         return_estimator=model_prop is not None,
#     )
#     if model_prop is not None:
#         if model_step is not None:
#             values = [
#                 getattr(m[model_step], model_prop) for m in cv_results["estimator"]
#             ]
#         else:
#             values = [getattr(m, model_prop) for m in cv_results["estimator"]]
#         print(f"Mean model.{model_prop} = {np.mean(values)}")
#     mae = -cv_results["test_neg_mean_absolute_error"]
#     rmse = -cv_results["test_neg_root_mean_squared_error"]
#     print(
#         f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
#         f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
#     )


# evaluate(gbrt, X, y, cv=ts_cv, model_prop="n_iter_")